## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description
0,0,Vercheres,45.78,-73.35,59.00,56,66,1.99,broken clouds
1,1,Chuy,-33.70,-53.46,54.27,85,98,15.32,light rain
2,2,Vercheres,45.78,-73.35,59.00,56,66,1.99,broken clouds
3,3,Chuy,-33.70,-53.46,54.27,85,98,15.32,light rain
4,4,Sandnessjoen,66.02,12.63,55.40,66,75,3.36,broken clouds
5,5,Tuatapere,-46.13,167.68,62.01,62,49,13.00,scattered clouds
6,6,Maoming,21.65,110.90,75.45,88,80,3.96,light rain
7,7,Riachao,-7.36,-46.62,91.74,21,88,2.98,overcast clouds
8,8,Bethel,41.37,-73.41,64.99,39,20,10.29,few clouds
9,9,Rikitea,-23.12,-134.97,73.29,77,1,15.97,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description
6,6,Maoming,21.65,110.90,75.45,88,80,3.96,light rain
10,10,Saint George,37.10,-113.58,90.00,6,1,4.05,clear sky
14,14,Luau,-10.71,22.22,75.22,31,37,5.59,scattered clouds
16,16,Kapaa,22.08,-159.32,84.20,66,40,11.41,scattered clouds
17,17,Kavaratti,10.57,72.64,80.71,71,96,11.03,overcast clouds
22,22,Butaritari,3.07,172.79,80.35,75,100,5.95,overcast clouds
26,26,Acajutla,13.59,-89.83,84.20,58,20,5.82,few clouds
31,31,Kudahuvadhoo,2.67,72.89,82.20,72,51,13.40,broken clouds
33,33,Conde,-7.26,-34.91,78.80,74,20,14.99,few clouds
34,34,Monrovia,6.30,-10.80,78.80,94,40,9.89,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.count()


City_ID                199
City                   199
Lat                    199
Lng                    199
Max Temp               199
Humidity               199
Cloudiness             199
Wind Speed             199
Weather description    199
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)].dropna()


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Max Temp", "Weather description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Max Temp,Weather description,Lat,Lng,Hotel Name
6,Maoming,75.45,light rain,21.65,110.90,
10,Saint George,90.00,clear sky,37.10,-113.58,
14,Luau,75.22,scattered clouds,-10.71,22.22,
16,Kapaa,84.20,scattered clouds,22.08,-159.32,
17,Kavaratti,80.71,overcast clouds,10.57,72.64,
22,Butaritari,80.35,overcast clouds,3.07,172.79,
26,Acajutla,84.20,few clouds,13.59,-89.83,
31,Kudahuvadhoo,82.20,broken clouds,2.67,72.89,
33,Conde,78.80,few clouds,-7.26,-34.91,
34,Monrovia,78.80,scattered clouds,6.30,-10.80,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

KeyboardInterrupt: 

In [47]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df

,City,Max Temp,Weather description,Lat,Lng,Hotel Name
6,Maoming,75.45,light rain,21.65,110.90,Jinhui Hotel
10,Saint George,90.00,clear sky,37.10,-113.58,Best Western Plus Abbey Inn
14,Luau,75.22,scattered clouds,-10.71,22.22,Kimbo Hotel
16,Kapaa,84.20,scattered clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
17,Kavaratti,80.71,overcast clouds,10.57,72.64,Hotels in Lakshadweep Islands
...,...,...,...,...,...,...
677,Wagar,88.56,few clouds,16.15,36.20,
680,Waddan,87.98,scattered clouds,29.16,16.14,محل الشريف محمد العايب
685,Marsh Harbour,83.25,light rain,26.54,-77.06,Abaco Beach Resort
688,Jacobabad,79.83,clear sky,28.28,68.44,Sardar Hotel


In [48]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_df.to_csv(output_data_file, index_label="City_ID")

In [52]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Weather description</dt><dd>{Weather description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_df[["Lat", "Lng"]]

In [53]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))